In [1]:
import numpy as np
from numpy.linalg import inv
import numpy.linalg as npl
import numpy.random as npr
import pandas as pd
import cufflinks as cf
import seaborn as sns
import matplotlib.pyplot as plt
from timeit import Timer
from scipy import linalg
from scipy.sparse import diags
from sympy import *
import sympy as sym
sym.init_printing()
from IPython.display import display, Math
from numpy.linalg import *

In [2]:
from methods import jacobi,jacobi_2, sor,SOR_num, gauss_seidel,gauss_seidel_num, gauss, comp_w, error
from check_matrix import characteristics, isDDM, create_block_triag, runntime_compare_df

In [3]:
def matrix_setup(n):
    """
    This function sets up the matrices A and b using the first order central differences
    stencil to discretise Poisson's equation in 2D.

    """
    N = n**2 # Number of points
    h = 1./(n+1) # gridspacing
    A = np.zeros([N, N]) # initialise A

    #Diagonals
    lead_diag = np.diag(np.ones(N)*-4, 0)
    outer_diags = np.ones(N-1)
    for i in range(n-1, N-1, n):
        outer_diags[i] = 0
    outer_diags = np.diag(outer_diags, 1) + np.diag(outer_diags, -1)

    #Diagonals dependent on n
    n_diags = np.diag(np.ones(N-n), n) + np.diag(np.ones(N-n), -n)

    #Populate A matrix
    A += lead_diag + outer_diags + n_diags
    A = A/(h**2)
    #Populate the RHS b matrix
    b=np.zeros(N)
    b[int((N-1)/2)]=2

    return A,b # The matrix problem A.u = b can now be solved to give a solution to the PDE
n=5
A, b = matrix_setup(n)

In [80]:
def create_mat(n,add=4):
    from scipy.sparse import random
    from scipy.stats import rv_continuous
    from numpy.random import default_rng

    class CustomDistribution(rv_continuous):
        def _rvs(self,  size=None, random_state=None):
            return random_state.standard_normal(size)
    rng = default_rng()
    X = CustomDistribution(seed=rng)
    Y = X()  # get a frozen version of the distribution
    S = random(n, n, density=0.25, random_state=rng, data_rvs=Y.rvs)
    A = S.A

    #A=np.random.randint(low=-20,high=20,size=(n,n))
    for i in range(n):
        for j in range(n):
            if i == j:
                A[i][j]+= sum(abs(A[i]))
                A[i][j]+= 1

    #A = np.triu(A,k=0)

    B = random(n, 1, density=1.0, random_state=rng, data_rvs=Y.rvs)
    b = B.A
    #b = np.ones((n))
    b=np.random.randint(low=-20,high=20,size=(n,1))
    if  characteristics(A) == 0:
        print("Determinante ist Null")

    return A, b

A, b = create_mat(n=10)
A

array([[ 6.21282359, -1.57600982,  1.16823743,  0.        ,  0.        ,
        -2.16554619,  0.        ,  0.        , -0.30303016,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.22382419,  2.16131746,  0.        ,  0.        ,
         0.19468313,  0.        ,  0.30987617, -0.43293397,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  2.00119666,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -1.00119666],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  2.03097246,
         0.        ,  0.        ,  0.        ,  0.        , -1.03097246],
       [ 0.024668  ,  0.        ,  0.        ,  0.        ,  0.        ,
         2.1444638 ,  0.        ,  0.        , -0.495837  ,  0.6239588 ],
       [ 0.        ,  0.        , -1.01981189,  0.        ,  2.71325695,
         0.        ,  5.23204228,  0.        

In [81]:
l1, l2, l3,l4, la = symbols("x1 x2 x3 x.. x")
l = Matrix([l1,l2,l3,l4])
r = Matrix(np.round(b,4))
a = Matrix(np.round(A,4))
display(Math('\ \\text{  }%s   \\text{*} %s \\text{= } %s' % (sym.latex(sym.simplify(a)) , (sym.latex(sym.simplify(l))), (sym.latex(sym.simplify(r))))))

<IPython.core.display.Math object>

In [82]:
mat, jac_it, gasei_it, sor_it, gael_it = [],[],[],[],[]
jac_time, gasei_time, sor_time_arr, gael_time = [],[],[],[]
jac_error, gasei_error, sor_error, gael_error = [],[],[],[]

normen = [None, 'inf', 0, 1, 2]  # 2-Norm, Maximumsnorm, sum(x!=0), 1-Norm
max_iter = 10000000
mat_gr = [80, 160, 320, 640, 1280]
rg = range(10,200,20)
rg2 = range(300,601,100)
test_set = [4,5,10,20, 22, 25]
test_set2 = [11,21,31,41,51]  #9,11,15,21,25,28,319,11,15,21,25,28,31
t3 = [50, 300, 500,1000]

In [83]:
for n in t3:
    #A = create_block_triag(n)
    #b = np.ones((n))
    #A, b =matrix_setup(n=n)
    A,b=create_mat(n=n)
    print(f"Lauf: {n}")
    w =comp_w(A) #1.25
    true_sol = npl.solve(A, b)
    jac_erg, k_jacobi = jacobi_2(A,b, iter=max_iter)
    t_j = Timer(lambda:jacobi_2(A,b, iter=max_iter))
    gauss_seidel_erg, k_gauss_seidel = gauss_seidel(A, b, iter=max_iter)
    gs_t = Timer(lambda:gauss_seidel(A,b, iter=max_iter))
    sor_erg, k_sor = sor(A, b,w, iter=max_iter)
    sor_t = Timer(lambda:SOR_num(A,b,w, iter=max_iter))
    gauss_elim_erg, c = gauss(A=A,B=b)
    t_ge = Timer(lambda:gauss(A,b))
    #print(round(sor_t.timeit(1),4))
    # iteration
    mat.append(n)
    jac_it.append(k_jacobi)
    gasei_it.append(k_gauss_seidel)
    sor_it.append(k_sor)
    gael_it.append(c)
    # error
    jac_error.append(error(jac_erg, true_sol))
    gasei_error.append(error(gauss_seidel_erg, true_sol))
    sor_error.append(error(sor_erg, true_sol))
    gael_error.append(error(gauss_elim_erg, true_sol))
    #print(norm(np.subtract(np.round(gauss_elim_erg,16) , np.round(true_sol,16)), ord=1)  )
    # timer
    jac_time.append("{:.3E}".format((t_j.timeit(1))))
    gasei_time.append("{:.3E}".format((gs_t.timeit(1))))
    sor_time_arr.append("{:.3E}".format((sor_t.timeit(1))))
    gael_time.append("{:.3E}".format((t_ge.timeit(1))))

Lauf: 50
Spektralradius der Jacobi-Matrix: 0.6363165753901004
omega = 1.1290325988457666


C:\Users\eaut2\anaconda3\lib\site-packages\numpy\linalg\linalg.py:2585: RuntimeWarning:

overflow encountered in reduce



KeyboardInterrupt: 

In [72]:
df = runntime_compare_df(mat, jac_it, gasei_it, sor_it, gael_it,jac_time, gasei_time, sor_time_arr, gael_time)

In [73]:
df

Jacobi            Gauß Seidel                    SOR  \
            Iterationen Zeit (sek) Iterationen Zeit (sek) Iterationen   
Matrixgröße                                                             
4                     4  3.159E-04           4  3.780E-04          14   
8                     8  2.911E-04           7  2.598E-04          11   
16                    9  2.715E-04           9  2.714E-04          11   
25                   10  3.610E-04           9  3.156E-04          10   
11                   10  7.075E-04          10  4.274E-04          13   
21                   10  2.273E-03           9  1.208E-03          10   
31                    9  1.062E-03           8  9.026E-04           9   
41                    9  2.853E-03           8  1.915E-03           9   
51                    9  1.149E-03           8  1.001E-03           8   
50                    9  1.883E-03           8  1.455E-03           9   
300                   7  2.347E-01           6  2.529E-02           6   
500                   6  1.586E-01           6  3.885E-02           6   
1000                  6  5.312E-01           6  3.712E-01           6   

                       Gauß Eliminierung             
            Zeit (sek)       Iterationen Zeit (sek)  
Matrixgröße                                          
4            2.458E-04                 6  1.262E-04  
8            3.324E-04                28  1.652E-04  
16           5.364E-04               120  4.972E-04  
25           9.245E-04               300  1.181E-03  
11           4.762E-04                55  3.375E-04  
21           4.244E-03               210  2.468E-03  
31           8.979E-03               465  6.530E-03  
41           3.412E-03               820  9.665E-03  
51           4.873E-03              1275  9.846E-03  
50           6.708E-03              1225  1.671E-02  
300          4.398E-02             44850  4.038E-01  
500          3.313E-02            124750  6.372E-01  
1000         9.149E-02            499500  2.548E+00

In [74]:
df_error = runntime_compare_df(mat, jac_it, gasei_it, sor_it, gael_it,
                               jac_time=jac_error, gasei_time=gasei_error, sor_time_arr=sor_error, gael_time=gael_error, label = "Fehler")

In [75]:
df_error

Jacobi            Gauß Seidel                    SOR  \
            Iterationen     Fehler Iterationen     Fehler Iterationen   
Matrixgröße                                                             
4                     4  1.135E-01           4  4.163E-17          14   
8                     8  9.320E-01           7  7.980E-17          11   
16                    9  3.101E-01           9  2.555E-12          11   
25                   10  2.986E-01           9  7.874E-12          10   
11                   10  6.427E-01          10  1.197E-16          13   
21                   10  3.247E-01           9  3.794E-12          10   
31                    9  2.710E-01           8  2.415E-11           9   
41                    9  2.827E-01           8  1.267E-11           9   
51                    9  3.510E-01           8  1.188E-11           8   
50                    9  1.808E-01           8  1.305E-11           9   
300                   7  1.179E-01           6  1.778E-11           6   
500                   6  9.767E-02           6  2.768E-12           6   
1000                  6  6.226E-02           6  2.605E-13           6   

                       Gauß Eliminierung             
                Fehler       Iterationen     Fehler  
Matrixgröße                                          
4            1.038E-10                 6  5.857E-01  
8            6.994E-11                28  1.955E+00  
16           7.813E-12               120  2.199E+00  
25           4.017E-11               300  1.849E+00  
11           8.457E-12                55  2.137E+00  
21           3.499E-11               210  2.980E+00  
31           3.305E-11               465  2.249E+00  
41           8.102E-12               820  2.536E+00  
51           1.908E-11              1275  2.248E+00  
50           7.485E-12              1225  2.034E+00  
300          2.131E-11             44850  2.162E+00  
500          3.408E-12            124750  2.127E+00  
1000         2.636E-13            499500  2.081E+00

In [ ]:
plt.figure(figsize=(10,6))
#plt.plot(mat, [float(e)for e in jac_time], label= "JAC",  linewidth=3)
#plt.plot(mat, [float(e)  for e in gasei_time], label= "GS",  linewidth=3)
plt.plot( mat, [float(e) for e in sor_time_arr], label= "SOR",  linewidth=3,c="green")
plt.plot( mat, [float(e) for e in gael_time],  label= "Gauß Elimination",  linewidth=3,c="red")

plt.title('Laufzeitvergleich', fontweight='bold', fontsize=17, pad=20)
plt.xticks(rotation='0', fontsize=14)
plt.yticks(rotation='0', fontsize=14)
plt.xlabel('Matrixgröße (nxn)', fontsize=14, labelpad=15, weight ='bold')
plt.ylabel('Laufzeit (s)', fontsize=14, labelpad=15, weight ='bold')
plt.legend(frameon=True,loc='upper left', shadow=True)
plt.tight_layout(h_pad=3.0)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(mat, [float(e)for e in jac_error], label= "JAC",  linewidth=3)
plt.plot(mat, [float(e)  for e in gasei_error], label= "GS",  linewidth=3)
plt.plot( mat, [float(e) for e in sor_error], label= "SOR",  linewidth=3,c="green")
plt.plot( mat, [float(e) for e in gael_error],  label= "Gauß Elimination",  linewidth=3, c="red")

plt.title('Fehlervergleich', fontweight='bold', fontsize=17, pad=20)
plt.xticks(rotation='0', fontsize=14)
plt.yticks(rotation='0', fontsize=14)
plt.xlabel('Matrixgröße (nxn)', fontsize=14, labelpad=15, weight ='bold')
plt.ylabel('Fehler (10E-6)', fontsize=14, labelpad=15, weight ='bold')
plt.legend(frameon=True,loc='upper left', shadow=True)
plt.tight_layout(h_pad=3.0)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
w= 40
#plt.bar(mat,jac_it, label= "JAC",width = w)
#plt.bar(mat, gasei_it, label= "GS",width = w)
#plt.bar( mat, sor_it, label= "SOR",width = w)
plt.bar( mat, gael_it,  label= "Gauß Elimination", width = w , color="red")

plt.title('Vergleich Iterationen', fontweight='bold', fontsize=17, pad=20)
plt.xticks(rotation='0', fontsize=14)
plt.yticks(rotation='0', fontsize=14)
plt.xlabel('Matrixgröße (nxn)', fontsize=14, labelpad=15, weight ='bold')
plt.ylabel('Iterationen', fontsize=14, labelpad=15, weight ='bold')
plt.legend(frameon=True,loc='upper left', shadow=True)
plt.show()

In [ ]:
df_error.plot.bar(figsize=(10,6))
plt.title('Vergleich Iterationen', fontweight='bold', fontsize=17, pad=20)
plt.xticks(rotation='0', fontsize=14)
plt.yticks(rotation='0', fontsize=14)
plt.xlabel('Matrixgröße', fontsize=14, labelpad=15, weight ='bold')
plt.ylabel('Iterationen (10E6)', fontsize=14, labelpad=15, weight ='bold')
plt.legend(frameon=True,loc='best', shadow=True, fontsize="large")
plt.tight_layout(h_pad=3.0)
plt.show()

In [ ]:
n = 200
A = create_block_triag(n)
b = np.ones((n))

true_sol = npl.solve(A, b)
sor_erg, k_sor = sor(A, b,w, iter=max_iter)

round(norm(np.subtract(sor_erg , true_sol)),5)

In [ ]:
n=20
A = create_block_triag(n)
b = np.ones((n))
true_sol = npl.solve(A, b)

jac_erg, k_jacobi = jacobi(A,b, iter=max_iter)
gauss_seidel_erg, k_gauss_seidel = gauss_seidel(A, b, iter=max_iter)
sor_erg, sor_time, k_sor = sor(A, b,w=1, iter=max_iter)
gauss_elim_erg,c = gauss(A,b)

In [ ]:
gs_fehler=np.round(gauss_seidel_erg[len(gauss_seidel_erg)-1]-true_sol,4)
jk_fehler=np.round(jac_erg[len(jac_erg)-1]-true_sol,4)
sor_fehler=np.round(sor_erg[len(sor_erg)-1]-true_sol,4)

jr = Matrix(jk_fehler)
gr = Matrix(gs_fehler)
sr = Matrix(sor_fehler)
#display(Math('\ \\text{ Jacobi : }%s   \\text{Gauss-Seidel :} %s \\text{SOR :} %s' % (sym.latex(sym.simplify(jr)) , (sym.latex(sym.simplify(gr))), (sym.latex(sym.simplify(sr)))  ))   )

In [ ]:
latex_name =f"time_table_m{n}.tex"
#time_df.to_latex("C:\\Users\\eaut2\\DataspellProjects\\Numerik-Projekt\\Test\\"+f"{latex_name}", index=True, bold_rows=True, caption="Beispieltext", position="h!", label="example")

In [ ]:
l1, l2, l3,l4, la = symbols("x1 x2 x3 x.. x")
l = Matrix([l1,l2,l3,l4])
r = Matrix(b)
a = Matrix(A)
#display(Math('\ \\text{  }%s   \\text{*} %s \\text{= } %s' % (sym.latex(sym.simplify(a)) , (sym.latex(sym.simplify(l))), (sym.latex(sym.simplify(r)))  ))   )

In [ ]:
dict = {'Jacobi Verfahren': jac_erg,
        'Gauss-Seidel Verfahren': gauss_seidel_erg,
        "SOR Verfahren":sor_erg
        }
#label = [f'x{i}' for i in range(0,n)]
label = [f'x0 - x{n-1} ']

for verfahren_name,data in dict.items():

    plt.figure(figsize=(9,6))
    plt.plot(data) # linestyle='',marker='.'
    #plt.axvline(x=k_sor+0.5,linestyle='--',c='red')
    plt.title(verfahren_name, fontweight='bold', fontsize=17, pad=20)
    plt.xticks(rotation='0', fontsize=14)
    plt.yticks(rotation='0', fontsize=14)
    plt.xlabel('Iteration', fontsize=14, labelpad=15, weight ='bold')
    plt.ylabel('Wert', fontsize=14, labelpad=15, weight ='bold')
    plt.legend(labels = label,frameon=True,loc='best', shadow=True, fontsize="large")
    plt.tight_layout(h_pad=3.0)
    plt.grid(True)
    plt.show()

Ideen: LGS von n = 2 bis n = 6 mio

Plot: Fehler, Wert,